### Notebook to test model for final performance metrics

In [6]:
import sys
import os
from matplotlib import pyplot as plt
from tqdm import tqdm
import torch
sys.path.append(os.path.dirname(os.path.abspath('.')))

from dev.utils import load_wandb_model

model, cfg, train_loader, val_loader, test_loader, topo_data, metadata = load_wandb_model('8mg9ekf1', name='last.ckpt', device='cuda')

Cached dataset does not match arguments, re-building the dataset...



Dataloaders and Topology Data Overview:
Batch size: 16
----------------------------------------------------------
Name                 | Samples      | Batches     
----------------------------------------------------------
Train Loader         | 6924         | 432         
Val Loader           | 989          | 61          
Test Loader          | 1978         | 123         

Topology Data:
Number of nodes: 100
Number of edges: 300
----------------------------------------------------------
Imported model from run "8mg9ekf1".
  | Name                | Type              | Params
----------------------------------------------------------
0 | criterion           | L1Loss            | 0     
1 | input_encoder_patch | Linear            | 576   
2 | input_encoder_node  | Linear            | 384   
3 | edge_encoder        | Linear            | 384   
4 | gnns                | ModuleList        | 223 K 
5 | U                   | ModuleList        | 37.4 K
6 | mixer_patch         | MLPMixerTempo

In [7]:
# Run test with Lightning trainer
from lightning import Trainer

trainer = Trainer(
    accelerator='auto',
    devices=1,
    logger=None,
    enable_checkpointing=False
)
torch.set_float32_matmul_precision('medium')

# Run test and get metrics
test_results = trainer.test(model, test_loader)


/mnt/cephfs/store/gr-mc2473/lc865/misc/condaforge/envs/ml-gnn/lib/python3.10/site-packages/lightning/fabric/plugins/environments/slurm.py:204: The `srun` command is available on your system but is not used. HINT: If your intention is to run Lightning on SLURM, prepend your python command with `srun` like so: srun python /mnt/cephfs/store/gr-mc2473/lc865/misc/condaforge/en ...
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3]


Testing DataLoader 0: 100%|██████████| 123/123 [00:12<00:00,  9.70it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     test/all-mae-12        1.1915227174758911
     test/all-mae-24        1.1910849809646606
     test/all-mae-36        1.1971192359924316
    test/all-mae-all         1.191590666770935
    test/all-mape-12         481.2224426269531
    test/all-mape-24         482.3765869140625
    test/all-mape-36         477.2601013183594
    test/all-mape-all        479.8969421386719
  test/all-missing_rate             0.0
    test/all-rmse-12        1.4928072690963745
    test/all-rmse-24        1.4923545122146606
    test/all-rmse-36        1.4999562501907349
    test/all-rmse-all       1.4938907623291016
        test/loss           0.68634986